Compression

In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.fft import fft2, ifft2
from PIL import Image as im

In [3]:
def convert_to_greyscale(input_data):
    array = np.array(input_data)
    grayscale_image = np.mean(array, axis=-1, keepdims=True).astype(np.uint8)
    return grayscale_image.squeeze()

In [4]:
def compress(arr):
    f_transform = fft2(arr)
    sorted_f = np.sort(np.abs(f_transform.reshape(-1)))

    keep = 0.01
    threshold = sorted_f[int(np.floor((1 - keep) * len(sorted_f)))]
    indices = np.abs(f_transform) > threshold
    f_low = f_transform * indices
    lowarr =  ifft2(f_low).real
    #print(np.array_equal(lowarr, arr))
    return lowarr

In [5]:
def compress_with_color(arr):
    carr = np.zeros_like(arr)
    for i in range(arr.shape[-1]):
        carr[:, :, i] = compress(arr[:, :, i])
    return carr

In [6]:
def mae(original, compressed):
    if original.shape != compressed.shape:
        raise ValueError("Matrices should have the same sizes")
    
    MAE = np.sum(np.abs(original - compressed))

    n = np.prod(original.shape)
    return MAE / n

In [7]:
def mse(original, compressed):
    if original.shape != compressed.shape:
        raise ValueError("Matrices should have the same sizes")
    
    MSE = np.sum((original - compressed)**2)

    n = np.prod(original.shape)

    return MSE / n

In [9]:
img = im.open('car.jpg')
#arr = convert_to_greyscale(img)
#img = im.fromarray(arr)
img.show()

arr = np.array(img)
arr = np.array(img) / 255.0  # Przeskaluj do zakresu [0, 1]
carr = compress_with_color(arr)
arr = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
carr = np.clip(carr * 255.0, 0, 255).astype(np.uint8)
print(np.array_equal(arr, carr))
print(f'MAE: {mae(arr, carr)}')
print(f'MSE: {mse(arr, carr)}')
img = im.fromarray(carr)

img.show()

False
MAE: 124.47796997070313
MSE: 76.3034075249566
